# Discretização
- **Transforma valores contínuos em dados categorizados**
    

In [1]:
# Importando o pandas
import pandas as pd

In [2]:
# Importando e visualoizando os dados
titanic = pd.read_csv('train2.csv')
titanic.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Titulos
0,0,3,male,22.0,1,0,7.2500,S,Mr
1,1,1,female,38.0,1,0,71.2833,C,Mrs


# KBinsDiscretizer

####  n_bins = Em quantas categorias vai dividir
#### Encode: Especifica como os bins são codificados após a transformação:

- "Onehot": Cria uma coluna para cada categoria e informa se ele pertence ou nao aquela categoria (0, 1)
- "Ordinal": Codifica os bins como números inteiros sequenciais

#### Strategy: A estratégia utilizada para definir os limites dos bins. 
- "Uniform": Todos os bins têm o mesmo tamanho. Se for de 0 a 200, 4 divisões = 50 para cada (Independente da distribuição, o que pode ter intervalores sem nenhum dados, podendo prejudicar a visualização em grafico por exemplo)
- "Quantile": Os bins têm a mesma quantidade de dados, ou seja, eles são definidos pelos quantis dos dados, garantindo que cada bin tenha aproximadamente o mesmo número de amostras
- "kmeans": Os bins são definidos usando o algoritmo K-Means, onde os centroides dos clusters são usados como limites dos bins.


In [3]:
from sklearn.preprocessing import KBinsDiscretizer

In [45]:
est = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='quantile')

### Fazendo fit e transformando os dados

In [46]:
est = est.fit(titanic[['Fare']])
titanic['Fare_disc'] = est.transform(titanic[['Fare']])

### Visualizando os dados gerados

In [58]:
titanic.Fare_disc.value_counts().sort_index() # Em ordem

Fare_disc
0.0    223
1.0    217
2.0    226
3.0    225
Name: count, dtype: int64

### Visualizando em detalhes as copntagens de cada intervalo

In [48]:
grupos = titanic.groupby('Fare_disc')['Fare'].agg(['min','max','count']).reset_index()
grupos

,Fare_disc,min,max,count
0,0.0,0.0000,7.8958,223
1,1.0,7.9250,14.4000,217
2,2.0,14.4542,30.6958,226
3,3.0,31.0000,512.3292,225


### Podemos utilizar o bin_edges para verificar os limites das categorias

In [53]:
est.bin_edges_[0][:5] ### Filtrando so os numeros

array([  0.    ,   7.9104,  14.4542,  31.    , 512.3292])

# Binarização
- **Transforma os dados em binários** (0 - se for menor ou igual e u 1 se o  valor for maior que o limite passado) 
    - **`threshold`**: limite usado para classificação dos valores
- **Podemos fazer a binarização para determinar se as pessoas estavam viajando sozinhas ou acompanhadas
    

In [62]:
from sklearn.preprocessing import Binarizer

### Visualizando os valores de Parch

In [63]:
titanic["Parch"].value_counts()

Parch
0    678
1    118
2     80
5      5
3      5
4      4
6      1
Name: count, dtype: int64

### Instanciando e fazendo fit

In [68]:
binari = Binarizer(threshold=0.5)
binari = binari.fit(titanic[['Parch']])

### Fazendo a transformação

In [69]:
titanic['Viagem_so'] = binari.transform(titanic[['Parch']])

### Visualizando resultado

In [70]:
titanic[['Parch','Viagem_so']].value_counts()

Parch  Viagem_so
0      0            678
1      1            118
2      1             80
3      1              5
5      1              5
4      1              4
6      1              1
Name: count, dtype: int64